In [28]:
from pathlib import Path
import pandas as pd
import json
from datasets import load_dataset

In [29]:
# xquad_task_description_en = """
# ### Human: Answer the question based on the given passage. Your answer must be a single entity, name, or number extracted directly from the passage. Begin your response with, 'Based on the passage, the answer to the question is', followed by the answer and a new line. Do not include any other information in your response.

# Passage:

# {context}

# Question:

# {question}

# ### Assistant:
# """

# # Respond with 'Based on the passage, the answer to the question is' followed by your answer on a new line.

xquad_task_description_en = """
### Human: Answer the question from the given passage. Your answer should be directly extracted from the passage, and it should be a single entity, name, or number, not a sentence.

Passage:

{context}

Question:

{question}

Note: Your answer should be directly extracted from the passage and be a single entity, name, or number. Provide the answer in quotations.### Assistant: Based on the passage, the answer to the question is "
"""

# fabian
xquad_task_description_de = """
### Human: Beantworte die Frage anhand des vorgegebenen Textes. Deine Antwort muss direkt aus dem Text entnommen sein und aus einer einzelnen Einheit, einem Namen oder einer Zahl bestehen. Deine Antwort darf nicht aus einem Satz bestehen.

Text:

{context}

Frage:

{question}

Hinweis: Deine Antwort muss direkt aus dem Text entnommen werden und aus einer einzigen Einheit, einem Namen oder einer Zahl bestehen. Setze die Antwort in Anführungs- und Schlusszeichen.### Assistant: Anhand des Textes lautet die Antwort auf die Frage „
"""

# janine
xquad_task_description_es = """
### Human: Responda a la pregunta a partir del pasaje dado. Tu respuesta debe ser extraída directamente del texto, y debe ser una sola entidad, nombre o número, no una frase.

Texto:

{context}

Pregunta:

{question}

Nota: Tu respuesta debe ser extraída directamente del texto y debe ser una sola entidad, nombre o número. Presenta la respuesta entre comillas.### Assistant: Basándose en el texto, la respuesta a la pregunta es «
"""

# anastassia
xquad_task_description_ru = """
### Human: Ответь на вопрос из приведенного отрывка. Твой ответ должен быть непосредственно извлечен из отрывка и содержать только один элемент, будь это предмет, имя или число. Ответ не должен быть целым предложением.

Отрывок:

{context}

Вопрос:

{question}

Внимание: Ваш ответ должен быть непосредственно взят из отрывка и представлять собой один предмет, имя или число. Укажите ответ в кавычках.### Assistant: Исходя из отрывка, ответ на вопрос: «
"""

# # andrianos
# xquad_task_description_el = """
# ### Human: απάντησε στην ερώτηση από το δοθέν απόσπασμα. Η απάντησή σου πρέπει να ληφθεί επί λέξει από το απόσπασμα, και πρέπει να είναι μια οντότητα, ένας αριθμός ή ένα όνομα, όχι μια πρόταση.

# Απόσπασμα:

# {context}

# Ερώτηση:

# {question}

# Σημείωση: Η απάντησή σου θα πρέπει να ληφθεί επί λέξει από το απόσπασμα και πρέπει να είναι μια οντότητα, ένα όνομα ή ένας αριθμός. Δώσε την απάντηση σε εισαγωγικά.
# ### Assistant: Δεδομένου του κειμένου, η απάντηση στην ερώτηση είναι «
# """

# cui
xquad_task_description_zh = """
### Human：请根据给定的段落回答问题。你的答案应直接提取自该段落，且应该为一个专有名词、名字或数字，而不是一个句子。

段落：

{context}

问题：

{question}

请注意：你的答案应直接摘自该段落， 且为一个专有名词，名字或数字。请在引号中给出答案。 ### Assistant：根据这段文字，该问题的答案是 "
"""


language_prompts = {
    'en': xquad_task_description_en,
    'es': xquad_task_description_es,
    'de': xquad_task_description_de,
    'ru': xquad_task_description_ru,
    # 'hi': xquad_task_description_hi,
    # 'el': xquad_task_description_el,
    'zh': xquad_task_description_zh
}




In [31]:

for lang_tag in language_prompts.keys():
    data = load_dataset('xquad', f'xquad.{lang_tag}')['validation']
    print(data)
    with open(f'data/xquad_dev_en_{lang_tag}.json', 'w', encoding='utf8') as en_outf:
        with open(f'data/xquad_dev_{lang_tag}_{lang_tag}.json', 'w', encoding='utf8') as x_outf:
            for i, row in enumerate(data):
                # print(row)
                context = row['context'].strip()
                question = row['question'].strip()
                answer = row['answers']['text'][0].strip()
                if len(row['answers']['text']) > 1:
                    print(f'Warning: {len(row["answers"]["text"])} answers found for {lang_tag} data.')

                en_formatted_item = {
                    'instruction': xquad_task_description_en.format(context=context, question=question).strip(),
                    'answer': answer,
                }

                x_formatted_item = {
                    'instruction': language_prompts[lang_tag].format(context=context, question=question).strip(),
                    'answer': answer,
                }

                if i == 0:
                    print(en_formatted_item)
                    print(x_formatted_item)


                en_outf.write(f'{json.dumps(en_formatted_item, ensure_ascii=False)}\n')
                x_outf.write(f'{json.dumps(x_formatted_item, ensure_ascii=False)}\n')
            
        print(f'Finished writing {lang_tag} data. Wrote {len(data)} items.')

Found cached dataset xquad (/home/user/kew/.cache/huggingface/datasets/xquad/xquad.en/1.0.0/39e1ff0497cbbfb79bbff61024031c10872bbd7c4fd8bc250207a965c39d3336)
100%|██████████| 1/1 [00:00<00:00, 316.46it/s]


Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 1190
})
{'instruction': '### Human: Answer the question from the given passage. Your answer should be directly extracted from the passage, and it should be a single entity, name, or number, not a sentence.\n\nPassage:\n\nThe Panthers defense gave up just 308 points, ranking sixth in the league, while also leading the NFL in interceptions with 24 and boasting four Pro Bowl selections. Pro Bowl defensive tackle Kawann Short led the team in sacks with 11, while also forcing three fumbles and recovering two. Fellow lineman Mario Addison added 6½ sacks. The Panthers line also featured veteran defensive end Jared Allen, a 5-time pro bowler who was the NFL\'s active career sack leader with 136, along with defensive end Kony Ealy, who had 5 sacks in just 9 starts. Behind them, two of the Panthers three starting linebackers were also selected to play in the Pro Bowl: Thomas Davis and Luke Kuechly. Davis compiled 5½ 

Found cached dataset xquad (/home/user/kew/.cache/huggingface/datasets/xquad/xquad.es/1.0.0/39e1ff0497cbbfb79bbff61024031c10872bbd7c4fd8bc250207a965c39d3336)
100%|██████████| 1/1 [00:00<00:00, 349.12it/s]


Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 1190
})
{'instruction': '### Human: Answer the question from the given passage. Your answer should be directly extracted from the passage, and it should be a single entity, name, or number, not a sentence.\n\nPassage:\n\n\ufeffLos Panthers, que además de liderar las intercepciones de la NFL con 24 y contar con cuatro jugadores de la Pro Bowl, cedieron solo 308 puntos en defensa y se sitúan en el sexto lugar de la liga. Kawann Short, tacle defensivo de la Pro Bowl, lideró al equipo con 11 capturas, 3 balones sueltos forzados y 2 recuperaciones. A su vez, el liniero Mario Addison, consiguió 6 capturas y media. En la línea de los Panthers, también destacó como ala defensiva el veterano Jared Allen ―5 veces jugador de la Pro Bowl y que fue el líder, en activo, de capturas de la NFL con 136― junto con el también ala defensiva Kony Ealy, que lleva 5 capturas en solo 9 partidos como titular. Detrás de ellos, Thoma

Found cached dataset xquad (/home/user/kew/.cache/huggingface/datasets/xquad/xquad.de/1.0.0/39e1ff0497cbbfb79bbff61024031c10872bbd7c4fd8bc250207a965c39d3336)
100%|██████████| 1/1 [00:00<00:00, 377.32it/s]


Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 1190
})
{'instruction': '### Human: Answer the question from the given passage. Your answer should be directly extracted from the passage, and it should be a single entity, name, or number, not a sentence.\n\nPassage:\n\nDie Verteidigung der Panthers gab nur 308 Punkte ab und belegte den sechsten Platz in der Liga, während sie die NFL mit 24 Interceptions in dieser Kategorie anführte und sich mit vier Pro Bowl-Selektionen rühmen konnte. Pro Bowl Defensive Tackle Kawann Short führte das Team mit 11 Sacks an, erzwang zudem drei Fumbles und erzielte zwei Fumble Recoverys. Mario Addison, ebenfalls Lineman, addierte 6½ Sacks hinzu. Die Panthers-Line präsentierte auch den erfahrenen Defensive End Jared Allen, einen 5-fachen Pro-Bowler, der mit 136 Sacks der aktive Anführer in der NFL-Kategorie Karriere-Sacks war, sowie den Defensive End Kony Ealy, der 5 Sacks in nur 9 Starts erzielte. Nach ihnen wurden zwei der d

Found cached dataset xquad (/home/user/kew/.cache/huggingface/datasets/xquad/xquad.ru/1.0.0/39e1ff0497cbbfb79bbff61024031c10872bbd7c4fd8bc250207a965c39d3336)
100%|██████████| 1/1 [00:00<00:00, 343.71it/s]


Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 1190
})
{'instruction': '### Human: Answer the question from the given passage. Your answer should be directly extracted from the passage, and it should be a single entity, name, or number, not a sentence.\n\nPassage:\n\n\ufeffЗащита Пэнтерс уступила всего 308 очков, заняв шестое место в лиге, а также лидировала в НФЛ по перехватам с 24 и похвасталась четырьмя попаданиями в Пробоул. Дифенсив тэкл Пробоула Кейван Шорт лидирует в команде с 11 мешками, а также обеспечил три потери мяча и получил два. Нападающий Марио Эдисон добавил 6½ мешков. Линия Пэнтерс также представила ди-энда-ветерана Джареда Аллена, пятикратного участника Пробоула, который был активным лидером по количеству мешков в карьере НФЛ в количестве 136, вместе с ди-эндом Кони Или, у которого было 5 мешков всего за 9 стартов. Позади них для участия в Пробоуле также были выбраны два из трех стартовых лайнбекеров Пэнтерс: Томас Дэвис и Люк Кикли. 

Found cached dataset xquad (/home/user/kew/.cache/huggingface/datasets/xquad/xquad.zh/1.0.0/39e1ff0497cbbfb79bbff61024031c10872bbd7c4fd8bc250207a965c39d3336)
100%|██████████| 1/1 [00:00<00:00, 363.55it/s]

Dataset({
    features: ['id', 'context', 'question', 'answers'],
    num_rows: 1190
})
{'instruction': '### Human: Answer the question from the given passage. Your answer should be directly extracted from the passage, and it should be a single entity, name, or number, not a sentence.\n\nPassage:\n\n黑豹队的防守只丢了 308分，在联赛中排名第六，同时也以 24 次拦截领先国家橄榄球联盟 (NFL)，并且四次入选职业碗。职业碗防守截锋卡万·肖特以 11 分领先于全队，同时还有三次迫使掉球和两次重新接球。他的队友马里奥·爱迪生贡献了 6½ 次擒杀。黑豹队的防线上有经验丰富的防守端锋贾里德·艾伦，他是五次职业碗选手，曾以 136 次擒杀成为 NFL 职业生涯中的活跃领袖。另外还有在 9 场首发中就拿下 5 次擒杀的防守端锋科尼·伊利。在他们身后，黑豹队的三名首发线卫中有两人入选了职业碗：托马斯·戴维斯和卢克·坎克利。戴维斯完成了 5½ 次擒杀、四次迫使掉球和四次拦截，而坎克利带领球队在擒抱 (118) 中迫使两次掉球并拦截了他自己的四次传球。卡罗莱纳的第二防线有职业碗安全卫科特·科尔曼和职业碗角卫约什·诺曼，科尔曼带领球队完成了职业生涯中高达七次拦截并同时贡献了 88 次擒抱，而诺曼在本赛季成长为一名封锁角卫并完成了四次拦截，其中两次被判触地得分。\n\nQuestion:\n\n黑豹队的防守丢了多少分？\n\nNote: Your answer should be directly extracted from the passage and be a single entity, name, or number. Provide the answer in quotations.### Assistant: Based on the passage, the answer to the question is "', 'answer': '308'}
{'instruct